1) 데이터에 대한 이해와 전처리

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorflow.keras.utils import to_categorical

In [ ]:
# 예제 문장 저장

text = """경마장에 있는 말이 뛰고 있다\n그의 말이 법이다\n가는 말이 고와야 오는 말이 곱다\n"""

In [ ]:
t= Tokenizer()
t.fit_on_texts([text])
vocab_size = len(t.word_index) + 1

# 케라스 토크나이저의 정수 인코딩은 인덱스가 1부터 시작하지만,
# 케라스 원-핫 인코딩에서 배열의 인덱스가 0부터 시작하기 때문에
# 배열의 크기를 실제 단어 집합의 크기보다 +1로 생성해야함

print('단어 집합의 크기: {}'.format(vocab_size))

단어 집합의 크기: 12


In [ ]:
print(t.word_index)

{'말이': 1, '경마장에': 2, '있는': 3, '뛰고': 4, '있다': 5, '그의': 6, '법이다': 7, '가는': 8, '고와야': 9, '오는': 10, '곱다': 11}


In [ ]:
# 훈련 데이터 만들기

In [ ]:
sequences = []
for line in text.split('\n'):    # \n 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0]
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]   # 길이가 2 이상인 gram들 저장
        sequences.append(sequence)

print('학습에 사용할 샘플의 개수: {}'.format(len(sequences)))

학습에 사용할 샘플의 개수: 11


In [ ]:
for i in sequences:
    print(i)

[2, 3]
[2, 3, 1]
[2, 3, 1, 4]
[2, 3, 1, 4, 5]
[6, 1]
[6, 1, 7]
[8, 1]
[8, 1, 9]
[8, 1, 9, 10]
[8, 1, 9, 10, 1]
[8, 1, 9, 10, 1, 11]


In [ ]:
# 가장 긴 샘플의 길이에 전체 샘플 길이 맞추기

max_len = max(len(l) for l in sequences) 
print('샘플 최대 길이 : {}'.format(max_len))

샘플 최대 길이 : 6


In [ ]:
# 전체 샘플 길이를 6으로 패딩

sequences = pad_sequences(sequences, maxlen = max_len, padding='pre')
print(sequences)

[[ 0  0  0  0  2  3]
 [ 0  0  0  2  3  1]
 [ 0  0  2  3  1  4]
 [ 0  2  3  1  4  5]
 [ 0  0  0  0  6  1]
 [ 0  0  0  6  1  7]
 [ 0  0  0  0  8  1]
 [ 0  0  0  8  1  9]
 [ 0  0  8  1  9 10]
 [ 0  8  1  9 10  1]
 [ 8  1  9 10  1 11]]


In [ ]:
# 각 샘플의 마지막 단어 레이블로 분리하기

sequences = np.array(sequences)

X = sequences[:,:-1]      # 각 리스트의 마지막 값을 제외하고 저장
y = sequences[:,-1]       # 각 리스트의 마지막 값만 저장

In [ ]:
print(X)

[[ 0  0  0  0  2]
 [ 0  0  0  2  3]
 [ 0  0  2  3  1]
 [ 0  2  3  1  4]
 [ 0  0  0  0  6]
 [ 0  0  0  6  1]
 [ 0  0  0  0  8]
 [ 0  0  0  8  1]
 [ 0  0  8  1  9]
 [ 0  8  1  9 10]
 [ 8  1  9 10  1]]


In [ ]:
print(y)

[ 3  1  4  5  1  7  1  9 10  1 11]


In [ ]:
# 레이블에 대해 one-hot encoding
y = to_categorical(y, num_classes = vocab_size)

In [ ]:
print(y)

[[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


2) 모델 설계하기

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length = max_len-1)) # 각 단어를 10차원으로 embedding, 레이블 분리 후 길이는 -1
model.add(SimpleRNN(32))   # 은닉 상태 크기
model.add(Dense(vocab_size, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(X,y,epochs = 200, verbose=2)

Epoch 1/200
1/1 - 1s - loss: 2.4973 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4844 - accuracy: 0.1818
Epoch 3/200
1/1 - 0s - loss: 2.4715 - accuracy: 0.0909
Epoch 4/200
1/1 - 0s - loss: 2.4587 - accuracy: 0.1818
Epoch 5/200
1/1 - 0s - loss: 2.4457 - accuracy: 0.1818
Epoch 6/200
1/1 - 0s - loss: 2.4325 - accuracy: 0.2727
Epoch 7/200
1/1 - 0s - loss: 2.4189 - accuracy: 0.2727
Epoch 8/200
1/1 - 0s - loss: 2.4049 - accuracy: 0.2727
Epoch 9/200
1/1 - 0s - loss: 2.3905 - accuracy: 0.3636
Epoch 10/200
1/1 - 0s - loss: 2.3754 - accuracy: 0.3636
Epoch 11/200
1/1 - 0s - loss: 2.3596 - accuracy: 0.3636
Epoch 12/200
1/1 - 0s - loss: 2.3430 - accuracy: 0.3636
Epoch 13/200
1/1 - 0s - loss: 2.3256 - accuracy: 0.3636
Epoch 14/200
1/1 - 0s - loss: 2.3073 - accuracy: 0.3636
Epoch 15/200
1/1 - 0s - loss: 2.2880 - accuracy: 0.3636
Epoch 16/200
1/1 - 0s - loss: 2.2676 - accuracy: 0.3636
Epoch 17/200
1/1 - 0s - loss: 2.2463 - accuracy: 0.3636
Epoch 18/200
1/1 - 0s - loss: 2.2239 - accuracy: 0.3636
E

In [ ]:
# 문장 생성 함수 만들기

def sentence_generation(model, t, current_word, n):
    init_word = current_word
    sentence = ''
    for _ in range(n):
        encoded = t.texts_to_sequences([current_word])[0]   # 현재 단어 정수 인코딩
        encoded = pad_sequences([encoded], maxlen = 5, padding = 'pre')   # 패딩
        result = model.predict_classes(encoded, verbose = 0) # 입력한 현재 단어 x 에 대해 다음 단어 예측, result에 저장
        for word, index in t.word_index.items():
            if index == result:
                break
        current_word = current_word + ' ' + word  # 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word  # 문장에 예측 단어를 추가하여 저장
    sentence = init_word + sentence
    return sentence


In [ ]:
print(sentence_generation(model, t, '경마장에', 4))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [ ]:
print(sentence_generation(model, t, '그의', 2))

그의 말이 법이다


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
print(sentence_generation(model, t, '가는', 5))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


가는 말이 고와야 오는 말이 곱다
